<a href="https://colab.research.google.com/github/667029/KVP10k/blob/main/LayoutLMv3_KVP10k_ny.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_______

In [1]:
!pip install -q transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64

In [2]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [3]:
import os              #navigere mapper og filer, hente filbaner
from PIL import Image  #åpne, vise og manipulere bilder
import json            #lese/skrive til JSON-filer
from transformers import LayoutLMv3Processor
import torch
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
base_path = "/content/drive/MyDrive/DAT255/KVP10k-dataset/kvp10k/"
print(os.listdir(base_path))

['train', 'test']


In [6]:
processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False) # <-- Viktig fordi vi allerede har utført OCR på bildet og har tekst og bboxes

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
# Mapping fra tekstlige BIO-labels til tall som modellen bruker
label_map = {
    "O": 0,
    "B-KEY": 1,
    "I-KEY": 2,
    "B-VALUE": 3,
    "I-VALUE": 4,
}

# Funksjon for å skalere bounding boxes til 0-1000 (som LayoutLMv3 krever)
def normalize_bbox(bbox, width, height):
  return [
      int(1000 * (bbox[0] /width)),
      int(1000 * (bbox[1] / height)),
      int(1000 * (bbox[2] / width)),
      int(1000 * (bbox[3] / height))
  ]


def assign_label_for_box(box, boxes, label_type):
  """Returnerer liste med (index, label) for tokens som overlapper box"""
  overlaps = []
  for i, token_box in enumerate(boxes):
    if box_overlap(box, token_box) > 0:
      overlaps.append(i)

  overlaps = sorted(overlaps)

  labeled = []
  for j, idx in enumerate(overlaps):
    tag = f"B-{label_type}" if j == 0 else f"I-{label_type}"
    labeled.append((idx, tag))

  return labeled


#Sjekker om OCR-boksen overlapper med GTS(key/value)-boksen.
#Ved overlapp hører de til hverandre.
def box_overlap(box1, box2):
  x0 = max(box1[0], box2[0])
  y0 = max(box1[1], box2[1])
  x1 = min(box1[2], box2[2])
  y1 = min(box1[3], box2[3])
  return max(0, x1 - x0) * max(0, y1 - y0)


# Funksjon for å generere BIO-labels fra gts (ground truth).
# Lager en BIO-label for hvert token basert på om det overlapper med en key- eller value-boks fra GTS.
# Matcher hvert token fra OCR (word + bbox) mot key/value-bbokser fra gts:
# --> Token overlapper en nøkkelboks: B-KEY eller I-KEY
# --> Token overlapper en verdiboks: B-VALUE eller I-VALUE
# --> Ellers: O
def iob_from_kvps(words, boxes, kvps):
  labels = ["O"] * len(words)

  #Gå igjennom alle key-value-pairs
  for kvp in kvps:
    if "key" in kvp and "bbox" in kvp["key"]:
      key_bbox = kvp["key"]["bbox"]
      for idx, tag in assign_label_for_box(key_bbox, boxes, "KEY"):
        labels[idx] = tag

    if "value" in kvp and "bbox" in kvp["value"]:
      value_box = kvp["value"]["bbox"]
      for idx, tag in assign_label_for_box(value_box, boxes, "VALUE"):
        labels[idx] = tag

  return labels

In [8]:
def load_example(doc_id, base_path):
  image_path = os.path.join(base_path, "images", f"{doc_id}.png")
  ocr_path = os.path.join(base_path, "ocrs", f"{doc_id}.json")
  gt_path = os.path.join(base_path, "gts", f"{doc_id}.json")

  image = Image.open(image_path).convert("RGB")

  with open(ocr_path, "r", encoding="utf-8") as f:
    ocr_data = json.load(f)

  with open(gt_path, "r", encoding="utf-8") as f:
    gt_data = json.load(f)

  page = ocr_data["pages"][0]
  width, height = page["width"], page["height"]

  words = [w["text"] for w in page["words"]]
  raw_boxes = [w["bbox"] for w in page["words"]]
  boxes = [normalize_bbox(b, width, height) for b in raw_boxes]

  kvps = gt_data["kvps_list"]
  string_labels = iob_from_kvps(words, raw_boxes, kvps)
  labels = [label_map[l] for l in string_labels]

  #Fyller på med padding, og truncation klipper av hvis sekvensen har for mange tokens, returnerer som PyTorch-tensor
  encoding = processor(image, words, boxes=boxes, word_labels=labels, padding="max_length", truncation=True, return_tensors="pt")

  return encoding, words, boxes, string_labels, image

In [9]:
from datasets import Dataset, DatasetDict
from tqdm import tqdm
import os

def load_split(split_name, base_path, limit=None):
    split_path = os.path.join(base_path, split_name)
    gts_dir = os.path.join(split_path, "gts")

    doc_ids = sorted([
        fname.replace(".json", "")
        for fname in os.listdir(gts_dir)
        if fname.endswith(".json")
    ])

    if limit:
        doc_ids = doc_ids[:limit]

    examples = []

    for doc_id in tqdm(doc_ids, desc=f"Laster {split_name}"):
        try:
            encoding, _, _, _, _ = load_example(doc_id, split_path)

            example = {
                "id": doc_id,
                "input_ids": encoding["input_ids"].squeeze(0),
                "attention_mask": encoding["attention_mask"].squeeze(0),
                "bbox": encoding["bbox"].squeeze(0),
                "labels": encoding["labels"].squeeze(0),
                "pixel_values": encoding["pixel_values"].squeeze(0),
            }

            examples.append(example)

        except Exception as e:
            print(f"Feil i {doc_id}: {e}")

    return Dataset.from_list(examples)


In [21]:
# Last inn begrenset antall dokumenter for utvikling/testing
raw_train_dataset = load_split("train", base_path, limit=300)
raw_test_dataset = load_split("test", base_path, limit=20)

# Kombiner i Hugging Face-format
dataset = DatasetDict({
    "train": raw_train_dataset,
    "test": raw_test_dataset,
})

# Angi hvilke kolonner som skal konverteres til PyTorch-tensorer
dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "bbox", "labels", "pixel_values"],
)

# Splitt treningssettet i train + eval (f.eks. 80/20)
split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]

print("\n")
print(f"Fullt datasett: {dataset}")
print(f"Train: {train_dataset}")
print(f"Eval: {eval_dataset}")


Laster train:   8%|▊         | 24/300 [00:05<00:58,  4.73it/s]

Feil i 00acf400cc61aff2e87253ee2a467bf507a666a81e33964f0cdee4b43e3774a9: image file is truncated


Laster test: 100%|██████████| 20/20 [00:07<00:00,  2.67it/s]




Fullt datasett: DatasetDict({
    train: Dataset({
        features: ['id', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pixel_values'],
        num_rows: 299
    })
    test: Dataset({
        features: ['id', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pixel_values'],
        num_rows: 20
    })
})
Train: Dataset({
    features: ['id', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pixel_values'],
    num_rows: 239
})
Eval: Dataset({
    features: ['id', 'input_ids', 'attention_mask', 'bbox', 'labels', 'pixel_values'],
    num_rows: 60
})


In [22]:
train_dataset.features

{'id': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'bbox': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'pixel_values': Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None)}

In [23]:
example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])
pixel_values torch.Size([3, 224, 224])


In [24]:
processor.tokenizer.decode(train_dataset[0]["input_ids"])

'<s> UNCLASSIFIED P-8A December 2018 SAR Program Information Program Name P-8A Poseidon Multi-Mission Maritime Aircraft (P-8A) DoD Component Navy Responsible Office CAPT Tony Rossi Phone: 301-757-5703 PMA-290, Maritime Patrol & Reconnaissance Aircraft Fax: 301-757-5681 Program Executive Office, Air Anti-Submarine Warfare, ’ Assault and Special Mission Programs DSN Phone: ~~ 757-5703 22347 Cedar Point Rd, Building 2185, STE 3190, Unit 6 DSN Fax: 757-5681 Patuxent River, MD 20670-1161 Date Assigned: March 30, 2016 anthony.rossi@navy.mil April 17, 2019 10:45:52 UNCLASSIFIED 6</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [25]:
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

<s> -100
 UNCLASSIFIED 0
 P 3
- -100
8 -100
A -100
 December 3
 2018 4
 SAR 4
 Program 0
 Information 0
 Program 1
 Name 2
 P 3
- -100
8 -100
A -100
 Pose 4
idon -100
 Multi 4
- -100
Mission -100
 Maritime 4
 Aircraft 4
 ( 4
P -100
- -100
8 -100
A -100
) -100
 Do 1
D -100
 Component 2
 Navy 3
 Respons 0
ible -100
 Office 0
 CAP 0
T -100
 Tony 0
 Rossi 0
 Phone 1
: -100
 301 3
- -100
757 -100
- -100
5 -100
703 -100
 PM 0
A -100
- -100
290 -100
, -100
 Maritime 0
 Patrol 0
 & 0
 Rec 0
onnaissance -100
 Aircraft 0
 F 1
ax -100
: -100
 301 3
- -100
757 -100
- -100
5 -100
681 -100
 Program 0
 Executive 0
 Office 0
, -100
 Air 0
 Anti 0
- -100
Sub -100
marine -100
 Warfare 0
, -100
 � 1
� 1
 Assault 0
 and 0
 Special 0
 Mission 0
 Programs 0
 D 2
SN -100
 Phone 2
: -100
  0
~~ -100
 7 3
57 -100
- -100
5 -100
703 -100
 223 0
47 -100
 Cedar 0
 Point 0
 Rd 0
, -100
 Building 0
 2 0
185 -100
, -100
 STE 0
 3 0
190 -100
, -100
 Unit 0
 6 0
 D 1
SN -100
 F 2
ax -100
: -100
 7 3
57 -100
- -100
5 -1

In [26]:
from evaluate import load
metric = load("seqeval")

In [27]:
import numpy as np

label_list = ["O", "B-KEY", "I-KEY", "B-VALUE", "I-VALUE"]
label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for i, label in enumerate(label_list)}


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from transformers import TrainingArguments

train_args = TrainingArguments(
    output_dir="test",  # Modell og checkpoints
    max_steps=1000,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir="./logs",         # 👈 Viktig for TensorBoard
    logging_steps=10,             # Hvor ofte den logger
    report_to="tensorboard"       # 👈 Skrur på TensorBoard logging
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
from transformers import Trainer
from transformers.data.data_collator import default_data_collator

trainer = Trainer (
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-30-0f46e3c32373>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer (


In [33]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1589: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,0.720700,0.785639,0.472196,0.373045,0.416805,0.773557
200,0.461800,0.540529,0.672574,0.593448,0.630538,0.839467
300,0.636300,0.420550,0.652740,0.709605,0.679986,0.859891
400,0.487100,0.419393,0.664143,0.718541,0.690272,0.860385
500,0.190300,0.477656,0.744800,0.693224,0.718087,0.851603
600,0.452800,0.428855,0.688841,0.717051,0.702663,0.866009
700,0.214600,0.507554,0.752969,0.708116,0.729854,0.860483
800,0.130100,0.415774,0.701936,0.755771,0.727859,0.875974
900,0.120000,0.472315,0.746544,0.723753,0.734972,0.867292
1000,0.117300,0.460558,0.731618,0.740879,0.736219,0.872521


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:1589: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


TrainOutput(global_step=1000, training_loss=0.3532902420163155, metrics={'train_runtime': 533.2377, 'train_samples_per_second': 3.751, 'train_steps_per_second': 1.875, 'total_flos': 525092432044032.0, 'train_loss': 0.3532902420163155, 'epoch': 8.333333333333334})